# SQLite Progress Prototype

This notebook is a **prototype** for tracking student progress using an SQLite database. It sets up the initial database schema and demonstrates how we might save and query study tasks generated in Notebook 03.

**Purpose of this notebook**
- Create a simple SQLite database (`progress.db`) to store daily tasks.  
- Define tables for storing tasks, days, and completion status.  
- Test basic insert and query operations to confirm the structure works.  

**Why 03.5?**  
This notebook is not the final version of progress tracking. Instead, it acts as an outline that will be expanded into **Notebook 04 (SQLite_Progress)**.  

In Notebook 04 we will add:
- Real study plans generated from Notebook 03.  
- Logic to update tasks when a student marks them complete.  
- Rules that prevent a student from moving to the next day until the current one is done.  

**Inputs**
- Example study plan (mock data or output from Notebook 03).  

**Outputs**
- `progress.db` (SQLite database) with prototype tables and example rows.  



In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:

import sqlite3
import pandas as pd
import os

#db_path ""=

def setup_db():
    """Create database tables"""
    conn = sqlite3.connect("tasks.db")
    conn.executescript("""
        CREATE TABLE IF NOT EXISTS progress (id INTEGER PRIMARY KEY, name TEXT);
        CREATE TABLE IF NOT EXISTS tasks (id INTEGER PRIMARY KEY, plan_id INTEGER, title TEXT, done BOOLEAN DEFAULT 0);
    """)
    conn.close()

def create_plan(name, tasks):
    """Create plan with tasks"""
    conn = sqlite3.connect("tasks.db")
    cursor = conn.cursor()
    
    cursor.execute("INSERT INTO plans (name) VALUES (?)", (name,))
    plan_id = cursor.lastrowid
    
    for task in tasks:
        cursor.execute("INSERT INTO tasks (plan_id, title) VALUES (?, ?)", (plan_id, task))
    
    conn.commit()
    conn.close()
    return plan_id

def toggle_task(task_id):
    """Toggle task completion"""
    conn = sqlite3.connect("tasks.db")
    conn.execute("UPDATE tasks SET done = NOT done WHERE id = ?", (task_id,))
    conn.commit()
    conn.close()

def get_progress(plan_id):
    """Get completion percentage"""
    conn = sqlite3.connect("tasks.db")
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*), SUM(done) FROM tasks WHERE plan_id = ?", (plan_id,))
    total, done = cursor.fetchone()
    conn.close()
    
    return int((done or 0) / total * 100) if total else 0

def show_plan(plan_id):
    """Show plan with progress"""
    conn = sqlite3.connect("tasks.db")
    cursor = conn.cursor()
    
    cursor.execute("SELECT name FROM plans WHERE id = ?", (plan_id,))
    name = cursor.fetchone()[0]
    
    cursor.execute("SELECT id, title, done FROM tasks WHERE plan_id = ?", (plan_id,))
    tasks = cursor.fetchall()
    conn.close()
    
    print(f"\n{name} ({get_progress(plan_id)}%)")
    for task_id, title, done in tasks:
        print(f"{'✅' if done else '⏳'} [{task_id}] {title}")

# Usage
setup_db()
plan = create_plan("My Progress", ["Task 1", "Task 2", "Task 3", "Task 4","Task 5", "Task 6", "Task 7"])
show_plan(plan)

toggle_task(1)  # Complete first task
show_plan(plan)


My Progress (0%)
⏳ [39] Task 1
⏳ [40] Task 2
⏳ [41] Task 3
⏳ [42] Task 4
⏳ [43] Task 5
⏳ [44] Task 6
⏳ [45] Task 7

My Progress (0%)
⏳ [39] Task 1
⏳ [40] Task 2
⏳ [41] Task 3
⏳ [42] Task 4
⏳ [43] Task 5
⏳ [44] Task 6
⏳ [45] Task 7


In [4]:
import sqlite3
import pandas as pd
import os

In [7]:
def generate_7_day_plan(plan_name_or_id):
    """Retrieve study plan created in notebook 3"""
    conn = sqlite3.connect("tasks.db")
    cursor = conn.cursor()
    
    # If searching by name
    if isinstance(plan_name_or_id, str):
        cursor.execute("SELECT id FROM plans WHERE name = ?", (plan_name_or_id,))
        result = cursor.fetchone()
        plan_id = result[0] if result else None
    else:
        plan_id = plan_name_or_id
    
    conn.close()
    return plan_id

# Use it like this:
study_plan_id = generate_7_day_plan("Generated Study Plan")
if study_plan_id:
    show_plan(study_plan_id)